<a href="https://colab.research.google.com/github/ISABEL021/EXERCICIO_CURSO/blob/main/trabalho_deep_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Treinamento do zero: Use o modelo VGG16 sem treinamento prévio, remova a camada mais recente (camada de previsão) e adicione um classificador. Treine o modelo do zero e avalie.



In [ ]:
# Importação de biblitecas
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
# Carregando dataset
dataset, info = tfds.load('beans', with_info=True, as_supervised=True, split=['train', 'validation', 'test'])


In [ ]:
# Ajustamento do tamanho da imagem / Pré processamento
IMG_SIZE = 224

def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.
    return image, label

# Aplicando o redimensionamento nas bases
train_data = dataset[0].map(preprocess_image).batch(32)
val_data = dataset[1].map(preprocess_image).batch(32)
test_data = dataset[2].map(preprocess_image).batch(32)

In [ ]:
# Definição do modelo
base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False)
x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(3, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.inputs, outputs=outputs)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# Compilação para definição de métricas
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Treinamento do modelo
history = model.fit(train_data, epochs=10, validation_data=val_data)

33/33 [==============================] - 2347s 71s/step - loss: 1.1317 - accuracy: 0.3588 - val_loss: 1.0969 - val_accuracy: 0.3308


In [ ]:
# Geração de métricas de perfomance
train_loss, train_acc = model.evaluate(train_data)
val_loss, val_acc = model.evaluate(val_data)
test_loss, test_acc = model.evaluate(test_data)

# Print dos resultados
print('Training accuracy:', train_acc)
print('Training loss:', train_loss)
print('Validation accuracy:', val_acc)
print('Validation loss:', val_loss)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

4/4 [==============================] - 84s 20s/step - loss: 1.0969 - accuracy: 0.3516
Training accuracy: 0.3462282419204712
Training loss: 1.0970712900161743
Validation accuracy: 0.3308270573616028
Validation loss: 1.0969380140304565
Test accuracy: 0.3515625
Test loss: 1.096907138824463


2) Usando rede pré-treinada como extrator de recursos: Use o mesmo modelo de Exp. 1, mas com o modelo VGG16 pré-treinado no ImageNet. Congele todas as camadas do ResNet50, treine e avalie o modelo.

In [ ]:
import numpy as np
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

In [ ]:
# Carregando os dataset and splitando em treino, validação e teste
train_ds, val_ds, test_ds = tfds.load(name="beans", split=["train", "validation", "test"], as_supervised=True)



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/beans/0.1.0.incomplete3ZNUC2/beans-train.tfrecord*...:   0%|          | 0/…

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/beans/0.1.0.incomplete3ZNUC2/beans-validation.tfrecord*...:   0%|         …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/beans/0.1.0.incomplete3ZNUC2/beans-test.tfrecord*...:   0%|          | 0/1…

Dataset beans downloaded and prepared to /root/tensorflow_datasets/beans/0.1.0. Subsequent calls will reuse this data.


In [ ]:
# Redimensionamento
IMG_SIZE = 224
BATCH_SIZE = 32

In [ ]:
def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.keras.applications.vgg16.preprocess_input(image)
    return image, label

train_data = train_ds.map(preprocess).batch(BATCH_SIZE)
val_data = val_ds.map(preprocess).batch(BATCH_SIZE)
test_data = test_ds.map(preprocess).batch(BATCH_SIZE)

In [ ]:
# Modelo VGG16 
base_model = VGG16(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

In [ ]:
# Ajuste de feição para rodar o modelo
base_model = tf.keras.applications.VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

train_features = np.concatenate([x for x, _ in train_ds.batch(32)], axis=0)
train_labels = np.concatenate([y for _, y in train_ds.batch(32)], axis=0)

val_features = np.concatenate([x for x, _ in val_ds.batch(32)], axis=0)
val_labels = np.concatenate([y for _, y in val_ds.batch(32)], axis=0)

train_features = base_model.predict(train_features)
val_features = base_model.predict(val_features)

5/5 [==============================] - 391s 74s/step


In [ ]:
test_features = np.concatenate([x for x, _ in test_ds.batch(32)], axis=0)
test_labels = np.concatenate([y for _, y in test_ds.batch(32)], axis=0)
test_features = base_model.predict(test_features)

4/4 [==============================] - 376s 94s/step


In [ ]:
# Congele todas as camadas do VGG16
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
#GlobalAveragePooling2D: Esta camada calcula a média de cada mapa de recursos na entrada e retorna um único valor de saída para cada recurso.
#Aplicação de camada totalmente conectada com 128 neurônios e ativação ReLU. 
# Denso: Esta é outra camada totalmente conectada com 3 neurônios e ativação softmax.
model = tf.keras.Sequential([
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [ ]:
# Compilação do modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Treinamento
history = model.fit(train_features, train_labels, epochs=20, validation_data=(val_features, val_labels))



Epoch 1/20
33/33 [==============================] - 1s 16ms/step - loss: 0.0060 - accuracy: 1.0000 - val_loss: 0.1306 - val_accuracy: 0.9474
Epoch 2/20
33/33 [==============================] - 1s 17ms/step - loss: 0.0051 - accuracy: 1.0000 - val_loss: 0.1469 - val_accuracy: 0.9624
Epoch 3/20
33/33 [==============================] - 1s 15ms/step - loss: 0.0053 - accuracy: 1.0000 - val_loss: 0.1387 - val_accuracy: 0.9474
Epoch 4/20
33/33 [==============================] - 1s 16ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 0.1307 - val_accuracy: 0.9474
Epoch 5/20
33/33 [==============================] - 1s 17ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 0.1381 - val_accuracy: 0.9549
Epoch 6/20
33/33 [==============================] - 1s 16ms/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.1315 - val_accuracy: 0.9549
Epoch 7/20
33/33 [==============================] - 0s 11ms/step - loss: 0.0037 - accuracy: 1.0000 - val_loss: 0.1350 - val_accuracy: 0.9549
Epoch 8/20
33

In [ ]:
# Geração de métricas de perfomance
train_loss, train_acc = model.evaluate(train_features)
val_loss, val_acc = model.evaluate(val_features)
test_loss, test_acc = model.evaluate(test_features)

# Print dos resultados
print('Training accuracy:', train_acc)
print('Training loss:', train_loss)
print('Validation accuracy:', val_acc)
print('Validation loss:', val_loss)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

4/4 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Training accuracy: 0.0
Training loss: 0.0
Validation accuracy: 0.0
Validation loss: 0.0
Test accuracy: 0.0
Test loss: 0.0


In [ ]:
val_loss = model.evaluate(val_features)
test_loss = model.evaluate(test_features)


4/4 [==============================] - 0s 8ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00




Fine-tuning the latest layers: take a pre-trained VGG16 model, unfreeze the last blocks acording to the following:
3-a) - unfreeze the last convolutional blocks (from "block5_conv1"), train and evaluate the model



In [ ]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras import optimizers

In [ ]:
# Carregando modelo VGG16
model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# # Congele todas as camadas do VGG16
for layer in model.layers:
    layer.trainable = False


In [ ]:
# Camadas de classificação
model = Sequential()
model.add(model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(3, activation='softmax'))

ValueError: ignored

In [ ]:
# Unfreeze the last convolutional blocks
for layer in model.layers[-6:]:
    layer.trainable = True

In [ ]:

# Compilação com rate baixo (lr  0.0001)
optimizer = optimizers.Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Treinamento
history = model.fit(train_features, train_labels, epochs=20, validation_data=(val_features, val_labels))


/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


NameError: ignored

In [ ]:
# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Unfreeze the last convolutional blocks
for layer in base_model.layers[-6:]:
    layer.trainable = True

In [ ]:
# Classificador da basede convolução
x = base_model.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.5)(x)
predictions = keras.layers.Dense(3, activation='softmax')(x)

In [ ]:
# Compilação
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Treinamento
history = model.fit(train_data.shuffle(1000).batch(32), epochs=10, validation_data=val_data.batch(32))


Epoch 1/10


ValueError: ignored

In [ ]:
# Métricas
train_loss, train_acc = model.evaluate(train_data)
val_loss, val_acc = model.evaluate(val_data)
test_loss, test_acc = model.evaluate(test_data)

print('Training accuracy:', train_acc)
print('Training loss:', train_loss)
print('Validation accuracy:', val_acc)
print('Validation loss:', val_loss)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

3-b) - unfreeze the last convolutional blocks (from "block4_conv1"), train and evaluate the model.

In [ ]:
# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# unfreeze the last convolutional blocks (from "block4_conv1")
for layer in base_model.layers[15:]:
    layer.trainable = True

In [ ]:
# Camadas de classificação
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

NameError: ignored

In [ ]:
# Compilação
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Treinamento
history = model.fit(train_data.shuffle(1000).batch(32), epochs=10, validation_data=val_data.batch(32))


In [ ]:
# Mpetricas
train_loss, train_acc = model.evaluate(train_data)
val_loss, val_acc = model.evaluate(val_data)
test_loss, test_acc = model.evaluate(test_data)

# Print resultados
print('Training accuracy:', train_acc)
print('Training loss:', train_loss)
print('Validation accuracy:', val_acc)
print('Validation loss:', val_loss)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

3-c) - unfreeze all convolutional blocks, train and evaluate the model.
